In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
import xarray as xr
import pandas as pd
import csv
import subprocess
import os
import sage_data_client
import urllib
import tempfile
from xmovie import Movie
import cftime


%matplotlib inline

In [ ]:
# https://portal.sagecontinuum.org/data-browser?nodes=W038&window=d&apps=air-quality%3A0.2.0

In [ ]:
dfaq = sage_data_client.query(
    start="2022-04-15 19:00", end="2022-04-15 23:55",
    filter={
        "name": "env.air_quality.conc",
        'vsn' : 'W038'
    }
)

In [ ]:
dfaq['timestamp'] = pd.to_datetime(dfaq['timestamp'])
dfaq = dfaq.set_index('timestamp')

In [ ]:
my_fig = plt.figure(figsize=[15,5])
dfaq.value["2022-04-15 20:00":"2022-04-15 23:55"].plot()
plt.ylabel('PM2.5 mg/m$^3$')

In [ ]:
#Thermal Camera

In [ ]:
df = sage_data_client.query(
    start="2022-04-15 19:00", end="2022-04-15 23:55",
    filter={
        "plugin": "mobotix-sampler.*",
        "vsn" : "V008"
    }
)

targets = []
times = []
for i in range(len(df)):
    if 'celsius' in df.iloc[i].value:
        targets.append(df.iloc[i].value)
        times.append(df.iloc[i].timestamp)


In [ ]:
df.iloc[-9210]

In [ ]:
def read_mob_from_sage(url, datet):
    tempf = tempfile.NamedTemporaryFile()
    try:
        urllib.request.urlretrieve(url, tempf.name)
        head = []
        with open(tempf.name, 'r') as f:
            reader = csv.reader(f, delimiter=';')
            for i in range(7):
                headers = next(reader)
                head.append(headers)
            _ = next(reader)
            data = np.expand_dims(np.flipud(np.array(list(reader)).astype(float)), axis=0)
        print(data.shape)
        tt = datet.to_pydatetime()
        ctime = cftime.datetime(tt.year, tt.month, tt.day, hour=tt.hour, minute=tt.minute, second=tt.second)

        ds = xr.Dataset({
        'thermalimage': xr.DataArray(
                    data   = data,   # enter data here
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': [ctime],
                             'y' : np.arange(data.shape[1]),
                             'x' : np.arange(data.shape[2])},
                    attrs  = {
                        '_FillValue': -999.9,
                        'units'     : 'celsius'
                        }
                    ),
                },
            attrs = {'Source': 'MOBOTIX M16 camera operated by Sage',
                    'URL' : url}
        )
    except urllib.error.HTTPError:
        ds="ERROR"
    
    return ds
    

In [ ]:
i = -1 #3000
myxr = read_mob_from_sage(targets[i], times[i])